In [1]:
import numpy as np
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
import pandas as pd
import os
global image_paths, target_size
basedir = '../Datasets/celeba'
images_dir = os.path.join(basedir,'img')
labels_filename = 'labels.csv'

In [2]:
import numpy as np
# from keras.preprocessing import image
from tensorflow.keras.preprocessing import image
import cv2
import dlib

# PATH TO ALL IMAGES

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('../shape_predictor_68_face_landmarks.dat')


# how to find frontal human faces in an image using 68 landmarks.  These are points on the face such as the corners of the mouth, along the eyebrows, on the eyes, and so forth.

# The face detector we use is made using the classic Histogram of Oriented
# Gradients (HOG) feature combined with a linear classifier, an image pyramid,
# and sliding window detection scheme.  The pose estimator was created by
# using dlib's implementation of the paper:
# One Millisecond Face Alignment with an Ensemble of Regression Trees by
# Vahid Kazemi and Josephine Sullivan, CVPR 2014
# and was trained on the iBUG 300-W face landmark dataset (see https://ibug.doc.ic.ac.uk/resources/facial-point-annotations/):
#     C. Sagonas, E. Antonakos, G, Tzimiropoulos, S. Zafeiriou, M. Pantic.
#     300 faces In-the-wild challenge: Database and results.
#     Image and Vision Computing (IMAVIS), Special Issue on Facial Landmark Localisation "In-The-Wild". 2016.


def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)


def run_dlib_shape(image):
    # in this function we load the image, detect the landmarks of the face, and then return the image and the landmarks
    # load the input image, resize it, and convert it to grayscale
    resized_image = image.astype('uint8')

    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    gray = gray.astype('uint8')

    # detect faces in the grayscale image
    rects = detector(gray, 1)
    num_faces = len(rects)

    if num_faces == 0:
        return None, resized_image

    face_areas = np.zeros((1, num_faces))
    face_shapes = np.zeros((136, num_faces), dtype=np.int64)

    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        temp_shape = predictor(gray, rect)
        temp_shape = shape_to_np(temp_shape)

        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)],
        #   (x, y, w, h) = face_utils.rect_to_bb(rect)
        (x, y, w, h) = rect_to_bb(rect)
        face_shapes[:, i] = np.reshape(temp_shape, [136])
        face_areas[0, i] = w * h
    # find largest face and keep
    dlibout = np.reshape(np.transpose(face_shapes[:, np.argmax(face_areas)]), [68, 2])

    return dlibout, resized_image

def extract_features_labels():
    """
    This funtion extracts the landmarks features for all images in the folder 'dataset/celeba'.
    It also extract the gender label for each image.
    :return:
        landmark_features:  an array containing 68 landmark points for each image in which a face was detected
        gender_labels:      an array containing the gender label (male=0 and female=1) for each image in
                            which a face was detected
    """
    image_paths = [os.path.join(images_dir, l) for l in os.listdir(images_dir)]
    target_size = None
    labels_file = open(os.path.join(basedir, labels_filename), 'r')
    lines = labels_file.readlines()
    gender_labels = {line.split('\t')[0] : int(line.split('\t')[3]) for line in lines[1:]}
    if os.path.isdir(images_dir):
        all_features = []
        all_labels = []
        for img_path in image_paths:
            #windows: file_name= img_path.split('.')[2].split('\\')[-1]
            #mac: file_name= img_path.split('.')[2].split('/')[-1]
            file_name= img_path.split('.')[2].split('\\')[-1]

            # load image
            img = image.img_to_array(
                image.load_img(img_path,
                               target_size=target_size,
                               interpolation='bicubic'))
            features, _ = run_dlib_shape(img)
            if features is not None:
                all_features.append(features)
                all_labels.append(gender_labels[file_name])

    landmark_features = np.array(all_features)
    gender_labels = (np.array(all_labels) + 1)/2 # simply converts the -1 into 0, so male=0 and female=1
    return landmark_features, gender_labels

In [3]:
#training:4795 testing:969
tr_X, y = extract_features_labels()
tr_Y = np.array([y, -(y - 1)]).T
basedir = '../Datasets/celeba_test'
images_dir = os.path.join(basedir,'img')
te_X, y = extract_features_labels()
te_Y = np.array([y, -(y - 1)]).T

In [4]:
def printOutput(m):
    print("accuracy", accuracy_score(list(zip(*te_Y))[0], m))
    print("precision", precision_score(list(zip(*te_Y))[0], m, average = None))
    print("recall", recall_score(list(zip(*te_Y))[0], m, average = None))

In [ ]:
"""
svm
accuracy 0.890608875128999
precision [0.88607595 0.89494949]
recall [0.88983051 0.89134809]

knn
k = 17
accuracy 0.8875128998968008
precision [0.89200864 0.88339921]
recall [0.875 0.89939638]

adaboost 38
accuracy 0.8875128998968008
precision [0.88865096 0.88645418]
recall [0.87923729 0.89537223]
"""

In [11]:
s = "0.87923729 0.89537223"
temp = s.split(" ")
temp = list(map(float, temp))
sum(temp)/len(temp)

0.8873047599999999

In [8]:
svmc = svm.SVC(kernel = "linear").fit(tr_X.reshape((len(tr_X), 68*2)), list(zip(*tr_Y))[0])
svmc_pred = svmc.predict(te_X.reshape((len(te_X), 68*2)))

In [9]:
printOutput(svmc_pred)

accuracy 0.890608875128999
precision [0.88607595 0.89494949]
recall [0.88983051 0.89134809]


In [5]:
aknn = []
def img_knn(training_images, training_labels, test_images, test_labels, k):
    
    classifier = KNeighborsClassifier(n_neighbors=k, algorithm = 'brute')
    
    scores = cross_val_score(classifier, tr_X.reshape((len(tr_X), 68*2)), list(zip(*tr_Y))[0], cv = 5)

    aknn.append([scores.mean(), scores.std()])


In [6]:
for i in range(1, 70):
    pred=img_knn(tr_X.reshape((len(tr_X), 68*2)), list(zip(*tr_Y))[0], te_X.reshape((len(te_X), 68*2)),list(zip(*te_Y))[0], i)

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

...

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

In [11]:
print(aknn)

[[0.821897810218978, 0.01337653197983279], [0.8350364963503649, 0.006897950118265363], [0.8458811261730969, 0.005764452538513133], [0.8548488008342023, 0.009497641323658573], [0.8550573514077163, 0.008994310178455835], [0.8565172054223149, 0.010419183313282038], [0.859436913451512, 0.009812963800641198], [0.8600625651720541, 0.01090462270064769], [0.8644421272158498, 0.0106340344391925], [0.8625651720542231, 0.010580730672755676], [0.8654848800834202, 0.011035459065962875], [0.8661105318039626, 0.01211748825554452], [0.8686131386861312, 0.009804095327251888], [0.8715328467153285, 0.010580730672755678], [0.870490093847758, 0.012467766571541316], [0.87007299270073, 0.009317333849296647], [0.8727841501564131, 0.012016567500219354], [0.8717413972888425, 0.013206373471522504], [0.8734098018769553, 0.011918441762026747], [0.8721584984358707, 0.011808456864977774], [0.8736183524504693, 0.012450312038873253], [0.8715328467153285, 0.013212958549703597], [0.8721584984358707, 0.014184505587597075

In [12]:
pd.DataFrame(aknn).to_csv("knn_output.csv")

In [13]:
aada = []
def img_ada(training_images, training_labels, test_images, test_labels, i):
    
    classifier = AdaBoostClassifier(n_estimators=i, random_state=0)
    
    scores = cross_val_score(classifier, tr_X.reshape((len(tr_X), 68*2)), list(zip(*tr_Y))[0], cv = 5)

    aada.append([scores.mean(), scores.std()])

In [14]:
for i in range(1, 200):
    pred=img_ada(tr_X.reshape((len(tr_X), 68*2)), list(zip(*tr_Y))[0], te_X.reshape((len(te_X), 68*2)),list(zip(*te_Y))[0], i)

In [16]:
print(aada)

[[0.8362877997914493, 0.001978484458916606], [0.8362877997914493, 0.001978484458916606], [0.8425443169968718, 0.0047556847762216335], [0.8460896767466111, 0.0059207797983642846], [0.8494264859228362, 0.00674492664391917], [0.8563086548488009, 0.0096565896999293], [0.859436913451512, 0.007673981018967322], [0.8619395203336809, 0.009795218824456652], [0.8619395203336808, 0.009728386894423893], [0.8621480709071949, 0.00816815684152918], [0.8642335766423358, 0.006866351463255227], [0.8613138686131385, 0.0070722940387125096], [0.8642335766423358, 0.009746253552073375], [0.8659019812304484, 0.007331969064128066], [0.87007299270073, 0.006777091474148893], [0.8706986444212722, 0.006821867459351893], [0.8715328467153285, 0.007414552172943155], [0.8717413972888425, 0.00734382350851264], [0.8711157455683003, 0.00888728915648075], [0.8715328467153285, 0.008300208025915518], [0.8742440041710114, 0.009008805450033465], [0.8736183524504693, 0.008141489591634276], [0.8790406673618353, 0.00810400801990

In [17]:
pd.DataFrame(aada).to_csv("ada_output.csv")

In [22]:
clf = AdaBoostClassifier(n_estimators=38, random_state=0)
clf.fit(tr_X.reshape((len(tr_X), 68*2)), list(zip(*tr_Y))[0])
clf_pred = clf.predict(te_X.reshape((len(te_X), 68*2)))

In [23]:
printOutput(clf_pred)

accuracy 0.8875128998968008
precision [0.88865096 0.88645418]
recall [0.87923729 0.89537223]


In [5]:
knn = KNeighborsClassifier(n_neighbors=41, algorithm = 'brute')
knn.fit(tr_X.reshape((len(tr_X), 68*2)), list(zip(*tr_Y))[0])
knn_pred = knn.predict(te_X.reshape((len(te_X), 68*2)))
printOutput(knn_pred)

accuracy 0.8875128998968008
precision [0.89200864 0.88339921]
recall [0.875      0.89939638]


D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
